In [1]:
push!(LOAD_PATH, "../src")
using Quaternions
using FixedSizeArrays
using RigidBodyDynamics

In [2]:
mechanism = Mechanism{Float64}("world")
world = root_body(mechanism)

RigidBody: "world"

In [3]:
body1 = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")));
joint1 = Joint("1", QuaternionFloating());
joint1ToWorld = rand(Transform3D{Float64}, joint1.frameBefore, world.frame)
body1ToJoint1 = rand(Transform3D{Float64}, body1.frame, joint1.frameAfter)
attach!(mechanism, world, joint1, joint1ToWorld, body1, body1ToJoint1);

In [4]:
joint2 = Joint("2", Revolute(rand(Vec{3, Float64})));
joint2ToBody1 = rand(Transform3D{Float64}, joint2.frameBefore, body1.frame)
body2frame = CartesianFrame3D("body2")
body2 = RigidBody(rand(SpatialInertia{Float64}, body2frame));
body2ToJoint2 = rand(Transform3D{Float64}, body2frame, joint2.frameAfter)
attach!(mechanism, body1, joint2, joint2ToBody1, body2, body2ToJoint2);

In [5]:
mechanism

Vertex: world (root)
  Vertex: body1, Edge: 1
    Vertex: body2, Edge: 2

In [6]:
x = MechanismState{Float64}(mechanism);

In [7]:
rand!(x)
x.v

DataStructures.OrderedDict{RigidBodyDynamics.Joint,Array{Float64,1}} with 2 entries:
  Joint "1": Quaternion f… => [0.4957103081796921,0.7362133824719759,0.51024729…
  Joint "2": Revolute joi… => [0.9134053056906435]

In [8]:
num_positions(x)

8

In [9]:
num_velocities(x)

7

In [10]:
cache = MechanismStateCache(mechanism, x);

In [11]:
relative_transform(cache, joint2.frameAfter, body1.frame)

Transform3D from "after_2" to "body1":
rotation: 3.467035593341797 rad about [0.02846897090283491,0.5421970150655782,0.8397689649836501], translation: Vec(0.016232907159798637,0.9766026621105754,0.6451421226490499)


In [12]:
center_of_mass(cache)

Point3D in "world": Vec(2.928913177065071,4.169104352580357,-1.5065568684617987)

In [13]:
mass(mechanism)

0.607819599235587

In [14]:
transform(cache, Point3D{Float64}(joint2.frameAfter), root_frame(mechanism))

Point3D in "world": Vec(1.9995718576550312,3.227326288167016,-0.43495194554787286)

In [15]:
relative_twist(cache, body2, body1)

Twist of "after_2" w.r.t "after_1" in "world":
angular: Vec(0.019065308097862776,0.8267445582611501,-0.005668835515041715), linear: Vec(0.34129897220581507,0.0030427511119177075,1.5916051821522001)

In [16]:
relative_twist(cache, body1, world) + relative_twist(cache, body2, body1) + relative_twist(cache, world, body2)

Twist of "world" w.r.t "world" in "world":
angular: Vec(0.0,0.0,0.0), linear: Vec(0.0,0.0,0.0)

In [17]:
geometric_jacobian(cache, path(mechanism, body2, world))

GeometricJacobian: body: "world", base: "after_2", expressed in "world":
6x7 Array{Float64,2}:
 -0.0208728   -0.8873     0.0307498  …  -0.0       -0.0        -0.0     
 -0.905123    -0.205551  -0.919561      -0.0       -0.0        -0.0     
  0.00620627   0.412853  -0.391743      -0.0       -0.0        -0.0     
 -0.373656     0.658911  -0.830655      -0.8873     0.0307498  -0.460167
 -0.00333122  -0.365921   0.496722      -0.205551  -0.919561    0.334898
 -1.7425       1.23394   -1.23119    …   0.412853  -0.391743   -0.822247

In [18]:
mass_matrix(cache)

7x7 Array{Float64,2}:
  0.414176      0.247419     -0.232334     …   0.000930538  -0.299911
 -0.0541988    -0.10687      -0.948785        -0.000934868   1.95404 
 -0.492117     -0.0764611     1.31723         -2.1684e-19   -0.544954
 -2.71051e-19  -0.00105546   -0.000930538      0.0          -0.311727
  0.00105546    1.35525e-19   0.000934868      0.0          -0.168312
  0.000930538  -0.000934868   1.0842e-19   …   0.00165913   -0.683606
 -0.299911      1.95404      -0.544954        -0.683606      1.28558 

In [19]:
A = momentum_matrix(cache)

MomentumMatrix expressed in "world":
6x7 Array{Float64,2}:
  0.14482       0.187092      0.430561     …   0.00305446   -1.47701  
  0.19852      -0.0211305    -1.36217         -0.00255864    2.23749  
 -0.59669      -0.204453      0.809629        -0.00275154    1.97899  
  0.000395748  -0.00136671   -0.000854413      0.000763475  -0.585557 
  0.000658926   9.61343e-5    0.000668395     -0.000555638   0.0100743
  0.0011786    -0.000332941   0.000750404  …   0.00136421   -0.500098 

In [20]:
A * velocity_vector(x)

Momentum expressed in "world":
angular: Vec(-0.9179043508505244,1.4298167247136506,1.7724215831567534), linear: Vec(-0.5347597894424065,0.010878996857875172,-0.45555251656184015)

In [21]:
accel = SpatialAcceleration(joint2.frameAfter, world.frame, joint2.frameAfter, rand(Vec{3, Float64}), rand(Vec{3, Float64}))
transform(cache, accel, world.frame)

SpatialAcceleration of "after_2" w.r.t "world" in "world":
angular: Vec(0.741705367141254,0.7852134707597934,0.05380511326249236), linear: Vec(1.383857752603486,-0.11891708904260989,-0.4360952235503222)

In [22]:
bias_acceleration(cache, body2)

SpatialAcceleration of "after_2" w.r.t "world" in "world":
angular: Vec(-0.41954710763480135,0.01329167105420942,0.5274490391732963), linear: Vec(0.9744584095659874,-0.8561032538308286,-8.550532502257425)

In [23]:
v̇ = [j::Joint => rand(num_velocities(j))::Vector{Float64} for j in joints(mechanism)]
τ = inverse_dynamics(cache, v̇)

7-element Array{Float64,1}:
 -8.69172  
  8.8375   
 -6.04003  
 -0.0642097
 -2.07861  
 -3.74292  
  5.36121  

In [26]:
RigidBodyDynamics.rand_chain_mechanism(Float64, QuaternionFloating, Revolute{Float64}, Revolute{Float64})

LoadError: LoadError: TypeError: subtype: expected Type{T}, got Tuple{DataType,DataType,DataType}
while loading In[26], in expression starting on line 1

In [25]:
RigidBodyDynamics.rand_tree_mechanism(Float64, [QuaternionFloating; [Revolute{Float64} for i = 1 : 10]]...)

LoadError: LoadError: TypeError: subtype: expected Type{T}, got Tuple{DataType,DataType,DataType,DataType,DataType,DataType,DataType,DataType,DataType,DataType,DataType}
while loading In[25], in expression starting on line 1